In [76]:
%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import os
from dotenv import load_dotenv, find_dotenv

from openai import OpenAI

In [78]:
load_dotenv()

True

In [79]:
# using groq api
open_ai_key = os.getenv('OPEN_API_KEY')
open_ai_base_url = "https://api.groq.com/openai/v1"
client = OpenAI(api_key=open_ai_key, base_url=open_ai_base_url)

In [122]:
chat_completion = client.chat.completions.create(
    model = 'llama3-8b-8192',
    messages=[{'role': 'user', 'content':'Hello, how are you?'}]
)

In [123]:
chat_completion.choices[0].message.content

"Hello! I'm just a language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?"

In [124]:
# creating a new agent class

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content":system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            model = 'llama3-8b-8192',
            messages=self.messages,
            temperature=0.7,
            max_tokens=200
        )
        return completion.choices[0].message.content

In [125]:
# creating a prompt for react agent

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [126]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [127]:
abot = Agent(prompt)

In [128]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I can use the average_dog_weight action to find the average weight of a toy poodle.

Action: average_dog_weight: Toy Poodle
PAUSE


In [129]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [132]:
next_prompt = "Observation: {}".format(result)

In [133]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs.'

In [90]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

### Part 2 - Restart the agent to clear the prev messages 

In [134]:
abot = Agent(prompt)

In [135]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'}]

In [136]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the weight of each dog breed and then combine them.\n\nAction: average_dog_weight: Border Collie\nPAUSE'

In [137]:
result = average_dog_weight("Border Collie")
next_prompt = "Observation: {}".format(result)

In [138]:
abot(next_prompt)

'Thought: Now I need to do the same for the Scottish Terrier, and then add the two weights together.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [139]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))

In [140]:
abot(next_prompt)

'Thought: Now I just need to add the two weights together.\n\nAction: calculate: 37 + 20\nPAUSE'

In [141]:
next_prompt = "Observation: {}".format(eval("37 + 20"))

In [142]:
abot(next_prompt)

'Answer: Your dogs combined weight is 57 lbs.'

In [143]:
abot(next_prompt)

'Answer: Your dogs combined weight is 57 lbs.'

### Part 3: creating a loop to avoid manual executions of action

In [144]:
# regex to select the appropriate action
action_re = re.compile('^Action: (\w+): (.*)$')

In [145]:
def query(question, max_turns=5):
    i = 0
    abot = Agent(prompt)
    next_prompt=question
    while i < max_turns:
        response = abot(next_prompt)
        i += 1
        actions = [action_re.match(a) for a in response.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [146]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
 -- running calculate 37 + 20
Observation: 57


### Hurray we have implemented an automated ai agent 